In [1]:
# Based on last year's winning submission:
# https://github.com/openproblems-bio/neurips2021_multimodal_topmethods/tree/main/src/predict_modality/methods/Guanlab-dengkw/run

import logging
import utils
import anndata as ad
import pandas as pd
import numpy as np
from scipy.sparse import csc_matrix

from sklearn.decomposition import TruncatedSVD
from sklearn.gaussian_process.kernels import RBF
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import KFold

logging.basicConfig(level=logging.INFO)

- Whole notebooks fits in about 1min for 10k rows. 
- 100k failed with an error: `Canceled future for execute_request message before replies were done`

In [9]:
class ExperimentParameters:
    MAX_ROWS = 10_000
    TECHNOLOGY: utils.Technology = utils.cite
    INPUT_PCA_DIMS = 128
    OUTPUT_PCA_DIMS = 128
    K_FOLDS = 3
    NP_RANDOM_SEED = 1000


np.random.seed(ExperimentParameters.NP_RANDOM_SEED)

In [10]:
logging.info('Reading `h5ad` files...')
# TODO: extract to method with params  (sparse/dense, technology)
input_train= pd.read_hdf(
    ExperimentParameters.TECHNOLOGY.train_inputs_path, 
    start=0, 
    stop=ExperimentParameters.MAX_ROWS
)
output_train= pd.read_hdf(
    ExperimentParameters.TECHNOLOGY.train_targets_path,
    start=0, 
    stop=ExperimentParameters.MAX_ROWS
)
input_test= pd.read_hdf(
    ExperimentParameters.TECHNOLOGY.test_inputs_path, 
    start=0, 
    stop=ExperimentParameters.MAX_ROWS
)

pred_dim_x = input_test.shape[0]
pred_dim_y = output_train.shape[1]


INFO:root:Reading `h5ad` files...


In [11]:
# Do PCA on all input data
input_pca_train = pd.concat(
    [input_train, input_test],
    axis=0,  # type: ignore
)

logging.info('Models using the Truncated SVD to reduce the dimension')

pca_input = TruncatedSVD(n_components=ExperimentParameters.INPUT_PCA_DIMS)
# TODO: float16 might be better, saw something in the forum
pca_features = pca_input.fit_transform(input_pca_train).astype(np.float32)
pca_train_input = pca_features[:len(input_train)] # First len(input_train) rows are input_train
pca_test_input = pca_features[len(input_train):] # Last len(input_test) rows are input_test
assert( len(pca_train_input) + len(pca_test_input) == len(pca_features))

del input_train
del input_test

INFO:root:Models using the Truncated SVD to reduce the dimension


In [12]:
# Also do PCA on output (needs to be de-reduced later)
pca_output = TruncatedSVD(n_components=ExperimentParameters.OUTPUT_PCA_DIMS)
pca_train_output = pca_output.fit_transform(output_train).astype(np.float32)

del output_train 

In [13]:
# Last year they found row-wise normalization was helpful, though they had
# to deal with more batch effects
def row_wise_std_scaler(M):
    """ 
    Standard scale values by row. 
    Sklearn StandardScaler has now row-wise option
    """
    std = np.std(M, axis=1).reshape(-1, 1)
    # Make any zero std 1 to avoid numerical problems
    std[std == 0] = 1
    mean = np.mean(M, axis=1).reshape(-1, 1)
    return (M - mean) / std

logging.info('Running row-wise normalization...')
# normalization across gene counts for a single cell.
# Possibly useful since we only care about correlation and not magnitude
# TODO: do we really want to normalize PCA input?
train_norm = row_wise_std_scaler(pca_train_input).astype(np.float32)
del pca_train_input 
test_norm = row_wise_std_scaler(pca_test_input).astype(np.float32)
del pca_test_input 

INFO:root:Running row-wise normalization...


In [14]:
logging.info('Running KRR model ...')
# TODO: research these more
SCALE = 10
ALPHA = .2
logging.info('Starting k-fold loop...')

kf = KFold(n_splits=ExperimentParameters.K_FOLDS)

def report_score(score):
    logging.info(f"Score: {score}")
    # TODO: add results writing
    
for fold_index, (train_index, test_index) in enumerate(kf.split(train_norm)):
    kernel = RBF(length_scale = SCALE)
    krr = KernelRidge(alpha=ALPHA, kernel=kernel)  # type: ignore
    logging.info(f'Fitting KRR fold {fold_index} of {ExperimentParameters.K_FOLDS}...')
    krr.fit(train_norm[train_index], pca_train_output[train_index])
    # TODO: review pca de-reduction
    Y_hat = krr.predict(train_norm[test_index]) @ pca_output.components_
    Y = pca_train_output[test_index] @ pca_output.components_
    score = utils.correlation_score(Y, Y_hat)
    report_score(score)

INFO:root:Running KRR model ...
INFO:root:Starting k-fold loop...
INFO:root:Fitting KRR fold 0 of 3...
INFO:root:Score: 0.8908717086652675
INFO:root:Fitting KRR fold 1 of 3...
INFO:root:Score: 0.8912362154462354
INFO:root:Fitting KRR fold 2 of 3...
INFO:root:Score: 0.8881603906429922
